# Data preparation
This notebook takes as input the original dataset and generates a clean dataset. It can run automatically or with some user inputs. User inputs generate a more consistent dataset, but for the dataset used it has no effect on model accuracy. 
In addition to the automatic mode, the numerical encoding and type of scaler are also parameters. 

In [1]:
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, RobustScaler, StandardScaler

auto_mode = True
scaler = RobustScaler()
numerical_null_values = 'knn'
categorical_null_values = 'knn'
output_file_path = 'Data/clean_dataset.csv'

## Parameter check

In [2]:
if auto_mode not in [True, False]:
    raise ValueError('"auto_mode" is not of type boolean')
if type(scaler) not in [MaxAbsScaler, MinMaxScaler, RobustScaler, StandardScaler]:
    raise ValueError('"scaler" is not of a scaler class')
if numerical_null_values not in ['mode', 'mean', 'knn']:
    raise ValueError('"numerical_null_values" has to be in [mode, mean, knn]')
if categorical_null_values not in ['knn', 'frequent']:
    raise ValueError('"numerical_null_values" has to be in [frequent, knn]')

## Import original dataset

In [3]:
import pandas as pd
import numpy as np

kidney_disease = pd.read_csv(r'Data/original_dataset.csv')
display(kidney_disease)

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,51,7200,5.9,no,no,no,good,no,no,notckd


## Rename columns

In [4]:
kidney_disease.columns = ['id', 'age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar',
       'red_blood_cells', 'pus_cell', 'pus_cell_clumps', 'bacteria',
       'blood_glucose_random', 'blood_urea', 'serum_creatinine', 'sodium',
       'potassium', 'hemoglobin', 'packed_cell_volume',
       'white_blood_cell_count', 'red_blood_cell_count', 'hypertension',
       'diabetes_mellitus', 'coronary_artery_disease', 'appetite',
       'peda_edema', 'aanemia', 'classification']

## Drop irrelevant columns

In [5]:
kidney_disease.drop(['id'], inplace=True, axis=1)

## Drop duplicated rows

In [6]:
kidney_disease.drop_duplicates()

,age,blood_pressure,specific_gravity,albumin,sugar,red_blood_cells,pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,...,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,appetite,peda_edema,aanemia,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47,6700,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54,7800,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49,6600,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51,7200,5.9,no,no,no,good,no,no,notckd


## Split Categorical and Numerical columns

In [7]:
numerical_cols = kidney_disease.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = kidney_disease.select_dtypes(include=['object']).columns.tolist()

print("Numerical Columns:\n", "  - ".join(numerical_cols))
print("\nCategorical Columns:\n", "  - ".join(categorical_cols))

Numerical Columns:
 age  - blood_pressure  - specific_gravity  - albumin  - sugar  - blood_glucose_random  - blood_urea  - serum_creatinine  - sodium  - potassium  - hemoglobin

Categorical Columns:
 red_blood_cells  - pus_cell  - pus_cell_clumps  - bacteria  - packed_cell_volume  - white_blood_cell_count  - red_blood_cell_count  - hypertension  - diabetes_mellitus  - coronary_artery_disease  - appetite  - peda_edema  - aanemia  - classification


### Convert false categorical to numerical

In [8]:
modified = False
for col in categorical_cols:
    if any(kidney_disease[col].dropna().apply(lambda x: x.isnumeric() if type(x)==str else False)):
        modified = True
        print(col+' was categorical')
        kidney_disease[col]=pd.to_numeric(kidney_disease[col], errors='coerce')

if modified:
    numerical_cols = kidney_disease.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = kidney_disease.select_dtypes(include=['object']).columns.tolist()

    print("\nNew numerical Columns:\n", "  - ".join(numerical_cols))
    print("\nNew categorical Columns:\n", "  - ".join(categorical_cols))

packed_cell_volume was categorical
white_blood_cell_count was categorical
red_blood_cell_count was categorical

New numerical Columns:
 age  - blood_pressure  - specific_gravity  - albumin  - sugar  - blood_glucose_random  - blood_urea  - serum_creatinine  - sodium  - potassium  - hemoglobin  - packed_cell_volume  - white_blood_cell_count  - red_blood_cell_count

New categorical Columns:
 red_blood_cells  - pus_cell  - pus_cell_clumps  - bacteria  - hypertension  - diabetes_mellitus  - coronary_artery_disease  - appetite  - peda_edema  - aanemia  - classification


## Categorical columns standardization
### Noise removing
Some of the categorical values contain misspelled data (example: '\tyes' for 'yes'). Here, we want to make sure all the misspells get replaced by their correct values.
#### User controlled way

In [9]:
if not auto_mode:
    for column in categorical_cols :
        distinct_values =  kidney_disease[column].dropna().unique()
        print(distinct_values)
        print('\n' + column + ':')
        for value in distinct_values:
            is_replaced = input("\t- '" + value + "' : replace? (y/N)")
            if is_replaced == 'y':
                replacement = input('\t  replace by: ')
                kidney_disease[column] = kidney_disease[column].replace(value, replacement)
        distinct_values =  kidney_disease[column].dropna().unique()
    
    print('\nBoolean columns now all contain only 2 distinct not-null values')

#### Automatic way

In [10]:
if auto_mode:
    kidney_disease[categorical_cols] = kidney_disease[categorical_cols].replace({'\t':'', ' ':''}, regex=True)
    for col in categorical_cols:
        kidney_disease[col]=kidney_disease[col].str.lower()

#### Check correction
In this dataset, all the categorical columns express a boolean value (examples: [yes, no], [present, notpresent]). We can check that all of the columns don't contain misspels by checking that they all contain only 2 distinct non-null values.

In [11]:
all_col_check = True
for column in categorical_cols :
    distinct_values =  kidney_disease[column].dropna().unique()
    if len(distinct_values) > 2:
        all_col_check = False
        print(column+' contains misspells : '+str(distinct_values))

if all_col_check:
    print('Check is cleared') 

Check is cleared


### String encoding
#### User controlled way
This way allows to keep some integrity in the data. For example, if yes is encoded as 1 in a column, it will also be encoded as 1 in another column, which we can't make sure of with automatic encoding.

In [12]:
if not auto_mode:
    boolean_columns = [x for x in categorical_cols if len(kidney_disease[x].dropna().unique())==2]
    for column in boolean_columns:
        distinct_values =  kidney_disease[column].dropna().unique()
        order = True
        print(column + ' : ' + str(distinct_values) + ' --> ' + str([order, not order]))
        is_reversed = input("Confirm order or reverse ? (C/r)")
        if is_reversed == 'r':
            order = not order
        kidney_disease[column] = kidney_disease[column].replace(distinct_values[0], int(order))
        print('\t- ' + distinct_values[0] + ' --> ' + str(order))
        kidney_disease[column] = kidney_disease[column].replace(distinct_values[1], int(not order))
        print('\t- ' + distinct_values[1] + ' --> ' + str(not order) + '\n')

#### Automatic way

In [13]:
if auto_mode:
    for col in categorical_cols:
        uniques=kidney_disease[col].dropna().unique()
        replace_in_order = [1, 0]
        if uniques[0]=='no' or uniques[0].startswith('not') or uniques[0].startswith('ab'):
            replace_in_order = [0, 1]
        kidney_disease[col] = kidney_disease[col].replace(uniques[0], replace_in_order[0])
        kidney_disease[col] = kidney_disease[col].replace(uniques[1], replace_in_order[1])

## Scaling

In [14]:
kidney_disease = pd.DataFrame(scaler.fit_transform(kidney_disease), columns=kidney_disease.columns)

## Null values handling
### Numerical columns
We have 3 different ways to replace null values in numerical columns:
- Mean
- Mode
- KNN prediction  

As our dataset has an important number of outliers, mode seems like a better choice than mean.

In [15]:
if numerical_null_values in ['mode', 'mean']:
    for col in numerical_cols:
        if numerical_null_values == 'mode':
            replacement = kidney_disease[col].mode()[0]
        else:
            replacement = kidney_disease[col].mean()
        kidney_disease[col] = kidney_disease[col].fillna(replacement)

### Categorical columns
- most frequent value
- KNN prediction  

In [16]:
if categorical_null_values=='frequent':
    for col in categorical_columns:
        most_frequent = kidney_disease[col].value_counts().idxmax()
        kidney_disease[col] = kidney_disease[col].fillna(most_frequent)

### KNN prediction
KNN prediction is more precise, but consumes more in both cases. As it is done for all the dataset at once, we do it last because otherwise other null values would also be replaced.

In [17]:
from sklearn.impute import KNNImputer

if 'knn' in [categorical_null_values, numerical_null_values]:
    imputer = KNNImputer(n_neighbors=5)
    kidney_disease = pd.DataFrame(imputer.fit_transform(kidney_disease), columns=kidney_disease.columns)
    
kidney_disease
categorical_cols

['red_blood_cells',
 'pus_cell',
 'pus_cell_clumps',
 'bacteria',
 'hypertension',
 'diabetes_mellitus',
 'coronary_artery_disease',
 'appetite',
 'peda_edema',
 'aanemia',
 'classification']

In [18]:
for col in categorical_cols:
    print(kidney_disease[col].dropna().unique())

[-0.2  0.  -0.4 -0.6 -1.  -0.8]
[ 0.  -1.  -0.4 -0.2 -0.6]
[0. 1.]
[0. 1.]
[1. 0.]
[1. 0.]
[0. 1.]
[ 0. -1.]
[0. 1.]
[0. 1.]
[ 0. -1.]


### No Null value check

In [19]:
if kidney_disease.isna().values.any():
    raise Exception('Dataset still contains null values')

## Save clean dataset into csv file

In [20]:
kidney_disease.to_csv(output_file_path, index=False)